In [1]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime
# from crawlers.models import *
# from crawlers.finlab.import_tools import *
from crawlers.finlab.data_process_tools import *

In [9]:
class CrawlStockMarginTransactionsTW:
    def __init__(self, date):
        self.date = date
        self.date_str = date.strftime("%Y%m%d")
        self.target_name = "台股融資交易資訊"
        self.sub_market = ["sii", "otc"]
        
    def crawl_sii(self):
        r= requests.get('http://www.twse.com.tw/exchangeReport/MI_MARGN?response=csv&date=' +self.date_str+ '&selectType=ALL')
        content = r.text.replace('=', '')
        lines = content.split('\n')
        lines = list(filter(lambda l:len(l.split('",')) > 10, lines))
        content = "\n".join(lines)
        if content == '':
            return None
        df = pd.read_csv(StringIO(content)).dropna(how='all', axis=1)
        if len(df)<1:
            return None
        df = df.astype(str)
        df = df.apply(lambda s: s.str.replace(',', ''))     
        df = df.rename(columns={'股票代號':'stock_id','股票名稱':'stock_name',
                                '買進':'mt_buy','賣出':'mt_sell',
                                '現金償還':'cash_redemption','前日餘額':'mt_balance_pd',
                                '今日餘額':'mt_balance_now','限額':'mt_quota',
                                '買進.1':'short_covering','賣出.1':'short_sale',
                                '現券償還':'stock_redemption','前日餘額.1':'ss_balance_pd',
                                '今日餘額.1':'ss_balance_now','限額.1':'ss_quota',
                                '資券互抵':'offset','註記':'note'})
        df.iloc[:,2:-1]=df.iloc[:,2:-1].apply(lambda s:pd.to_numeric(s,errors='coerce'))

        df['note']=[symbols_change(i,{'O':'停止融資','X':'停止融券','@':'融資分配','%':'融券分配','!':'停止買賣'}) for i in df['note']]
        df['mt_use_rate']=[round(v/q*100,2) if q>0 else 100 if v>0 else 0 for v,q in zip (df['mt_balance_now'],df['mt_quota'])]
        df['ss_use_rate']=[round(v/q*100,2) if q>0 else 100 if v>0 else 0 for v,q in zip (df['ss_balance_now'],df['ss_quota'])]
        df['date'] = pd.to_datetime(self.date)
        return df
    
    def crawl_otc(self):
        y = str(int(self.date.strftime("%Y")) - 1911)
        date_str = y + "/" + self.date.strftime("%m") + "/" + self.date.strftime("%d")
        r = requests.get('http://www.tpex.org.tw/web/stock/margin_trading/margin_balance/margin_bal_result.php?l=zh-tw&o=csv&d='+date_str+'&s=0,asc,0')
        df = pd.read_csv(StringIO(r.text), header=2).dropna(how='all', axis=1).dropna(how='any')
        if len(df)<10:
            return None
        df = df.astype(str).apply(lambda s: s.str.replace(',',''))
        df.columns=[col.replace(' ','') for col in df.columns]
        df = df.rename(columns={'代號':'stock_id','名稱':'stock_name',
                                '資買':'mt_buy','資賣':'mt_sell',
                                '現償':'cash_redemption','前資餘額(張)':'mt_balance_pd',
                                '資餘額':'mt_balance_now','資限額':'mt_quota',
                                '券買':'short_covering','券賣':'short_sale',
                                '券償':'stock_redemption','前券餘額(張)':'ss_balance_pd',
                                '券餘額':'ss_balance_now','券限額':'ss_quota',
                                '資券相抵(張)':'offset','備註':'note',
                                '資使用率(%)':'mt_use_rate','券使用率(%)':'ss_use_rate'})
        df=df.drop(columns=['資屬證金','券屬證金'])
        df.iloc[:,2:-1]=df.iloc[:,2:-1].apply(lambda s:pd.to_numeric(s,errors='coerce'))
        df['note']=[symbols_change(i,{'O':'停止融資','X':'停止融券','@':'融資分配','%':'融券分配','!':'停止買賣',
                                    '*':'融券餘額占融資餘額百分之六十以上者','A':'股價波動過度劇烈','B':'股權過度集中',
                                      'C':'成交量過度異常','D':'監視第二次處置'}) for i in df['note']]
        df['date'] = pd.to_datetime(self.date)     
        return df

    def crawl_main(self):
        try:
            df = pd.concat([self.crawl_sii(),self.crawl_otc()],sort=False)
        except ValueError:
            return None
        return df
    

import datetime
df=CrawlStockMarginTransactionsTW(datetime.datetime(2009,1,5)).crawl_main()
df

,stock_id,stock_name,mt_buy,mt_sell,cash_redemption,mt_balance_pd,mt_balance_now,mt_quota,short_covering,short_sale,stock_redemption,ss_balance_pd,ss_balance_now,ss_quota,offset,note,mt_use_rate,ss_use_rate,date
0,15,富邦,3,12,0,2122,2113,125000,0,0,0,0,0,125000,0,,1.69,0,2009-01-05
1,50,元大台灣50,2201,2643,2,33974,33530,297625,792,1016,12,8494,8706,297625,1682,,11.27,2.93,2009-01-05
2,51,元大中型100,28,15,0,1682,1695,13375,2,2,0,210,210,13375,1,,12.67,1.57,2009-01-05
3,52,富邦科技,5,0,0,58,63,7000,0,0,0,0,0,7000,0,,0.9,0,2009-01-05
4,53,元大電子,19,34,0,740,725,19247,1,1,0,68,68,19247,20,,3.77,0.35,2009-01-05
5,54,元大台商50,2,2,0,440,440,20406,0,0,0,17,17,20406,0,,2.16,0.08,2009-01-05
6,55,元大MSCI金融,256,408,2,8018,7864,60413,44,113,0,1537,1606,60413,186,,13.02,2.66,2009-01-05
7,56,元大高股息,0,3,0,585,582,25133,0,10,0,13,23,25133,1,,2.32,0.09,2009-01-05
8,57,富邦摩台,0,0,0,45,45,6756,0,0,0,0,0,6756,0,,0.67,0,2009-01-05
9,58,富邦發達,0,1,0,10,9,3174,0,0,0,0,0,3174,0,,0.28,0,2009-01-05


In [ ]:
df=CrawlStockTiiMarketReportTW(datetime.datetime(2020,4,29)).crawl_main()
add_to_sql(StockTiiMarketReportTW,df)

In [ ]:
#         符號說明： O：停止融資, X：停止融券, @：融資分配, %：融券分配, !：停止買賣
def symbols_change(word,target={}):
    if len(word)>0:
        for sym,value in target.items():
            if sym in word:
                word=word.replace(sym,'_'+value)
        if word[0] is '_':
            word=word[1:]
    return word
symbols_change('OX',{'O':'停止融資','X':'停止融券'})

In [ ]:
df.columns

In [ ]:
a={'name': 'Sky', 'attack': 100, 'hp': 500}.items()
a

In [ ]:
len(None)

In [ ]:
'    d'.replace(',','').replace(' ','')

In [6]:
len(df.columns)

19